In [0]:
# A-Z deep learning 
# Churn modelling 
# ANN - a classification model

In [0]:
import numpy as np
import pandas as pd

In [0]:
! pip install tensorflow==2.1.0

In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling (2).csv


In [0]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
dataset.shape

(10000, 14)

In [0]:
#checking for missing data in the dataset but considers only if the value is not avaiable.
print(dataset.isnull().sum())
"""incase the value is 0 then we need to look for another startegy
print((dataset[[1,2,3,4,5,6,7,8,9,10,11,12,13]]==0).sum())
we can replace this 0 values with Nan as numpy and pandas wont consider them for kinding out the mean, etc
dataset.replace(0,np.nan)  would do the work
we may also drop the rows with missing values as some algo in machine laenring give an erro if some null value is encountered
dataset.dropna(inplace=True)   ths piece of code will remove all the rows with value as null
"""

In [0]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [0]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [0]:
"""
impute means using a model to replace the missing values
NOT REQUIRED AS THERE ARE NO MSSING VALUES IN THE DATASET
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

we can also use pandas function to impute missing values for each column
dataset.fillna(dataset.mean(),  inplace=True)
"""

#splitting the dataset into input and output


In [0]:
#X = dataset.iloc[:,[3,-1]].values
#y = dataset.iloc[:,13].values

In [0]:
X = dataset.drop(['RowNumber', 'CustomerId','Surname','Exited'], axis = 1).values
Y = dataset['Exited'].values

The first step is to check for any missing data that can be seem from the dataset.info command

In [0]:
X.shape

(10000, 10)

we will be using label encoder to demarcate the different categories of data that we have in a column and then we will use one hot encoder for each category of data

In [0]:
X[:2,:]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58]],
      dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_x1 = LabelEncoder()
X[:,1] = labelencoder_x1.fit_transform(X[:,1])
labelencoder2 = LabelEncoder()
X[:,2] = labelencoder2.fit_transform(X[:,2])
print(X[:2,:])
X.shape

[[619 0 0 42 2 0.0 1 1 1 101348.88]
 [608 2 0 41 1 83807.86 1 0 1 112542.58]]


(10000, 10)

In [0]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1,2])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
X = ct.fit_transform(X)

print(X[:2,:])
X.shape

[[1.0 0.0 0.0 1.0 0.0 619 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 1.0 0.0 608 41 1 83807.86 1 0 1 112542.58]]


(10000, 13)

### Standard Scaling
Since the vlaues in the various column in X are having a huge range of values we dont want sice certain columns have higher numerical values hence they should have more weightage in deciding what the result of the midel should be

In [0]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(X)
print(X[:5,:])
X.shape

[[ 0.99720391 -0.57873591 -0.57380915  1.09598752 -1.09598752 -0.32622142
   0.29351742 -1.04175968 -1.22584767 -0.91158349  0.64609167  0.97024255
   0.02188649]
 [-1.00280393 -0.57873591  1.74273971  1.09598752 -1.09598752 -0.44003595
   0.19816383 -1.38753759  0.11735002 -0.91158349 -1.54776799  0.97024255
   0.21653375]
 [ 0.99720391 -0.57873591 -0.57380915  1.09598752 -1.09598752 -1.53679418
   0.29351742  1.03290776  1.33305335  2.52705662  0.64609167 -1.03067011
   0.2406869 ]
 [ 0.99720391 -0.57873591 -0.57380915  1.09598752 -1.09598752  0.50152063
   0.00745665 -1.38753759 -1.22584767  0.80773656 -1.54776799 -1.03067011
  -0.10891792]
 [-1.00280393 -0.57873591  1.74273971  1.09598752 -1.09598752  2.06388377
   0.38887101 -1.04175968  0.7857279  -0.91158349  0.64609167  0.97024255
  -0.36527578]]


(10000, 13)

#Splitting data into test and training set

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.1,random_state=0)
X_train.shape

(9000, 13)

# Building our Artificial Neural Network
We are making our model using the sequential model type. sequential model type help ua building a model layer by layer ie in a sequential manner.
we also use the dense layer type this means that all the nodes of the pervious layer are connected with the nodes of the current layer

Activation function helps in discovering the non linear relationships.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Till now we have just made the structure for the model.. now we will actually calcluate the weights for layers
optimizer is the gradient descent algo we use
if we have a binary output we use binary_crossentropy(log cost function), but if the output is having multiple categories then we use softmax function in the layer and the loss function will be categorical_crossentropy.

In [0]:
model = Sequential()
model.add(Dense(6, activation='relu', input_shape=(13,)))       # th first arg is the number of nodes in the layer, input size increases as we used onr=ehotencoder
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))                        #output layer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(X_train,y_train,batch_size=10, epochs=100)

Train on 9000 samples
Epoch 1/100
9000/9000 [==============================] - 2s 179us/sample - loss: 0.4929 - accuracy: 0.7840
Epoch 2/100
9000/9000 [==============================] - 1s 151us/sample - loss: 0.4341 - accuracy: 0.7980
Epoch 3/100
9000/9000 [==============================] - 1s 151us/sample - loss: 0.4218 - accuracy: 0.8170
Epoch 4/100
9000/9000 [==============================] - 1s 155us/sample - loss: 0.4095 - accuracy: 0.8264
Epoch 5/100
9000/9000 [==============================] - 1s 150us/sample - loss: 0.3881 - accuracy: 0.8376
Epoch 6/100
9000/9000 [==============================] - 1s 154us/sample - loss: 0.3680 - accuracy: 0.8480
Epoch 7/100
9000/9000 [==============================] - 1s 152us/sample - loss: 0.3572 - accuracy: 0.8537
Epoch 8/100
9000/9000 [==============================] - 1s 151us/sample - loss: 0.3520 - accuracy: 0.8552
Epoch 9/100
9000/9000 [==============================] - 1s 153us/sample - loss: 0.3481 - accuracy: 0.8592
Epoch 10/100
90

the accuracy of the training set is 85.78%
now we will check the accuracy for the test set

In [0]:
y_pred = model.predict(X_test)

evaluating the model for its accuracy and loss

In [0]:
loss, accuracy = model.evaluate(X_test,y_test,verbose=1)
print("test accuracy=",accuracy)

1000/1000 [==============================] - 0s 101us/sample - loss: 0.3277 - accuracy: 0.8600
test accuracy= 0.86
